In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle

from tqdm import tqdm
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint

from config.assistant import *
from config.site_config import *

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

5688


### 회사명에 부분적으로 포함되는 단어 필터링

In [5]:
# company_filtering_list_5 = ['온라인']

for filtering_word in filtering_corp_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['company']:
#                 print(job_list['company'])
                if job_list['status'] == 'wait':
                    job_list['status'] = 'close'
                    nCnt += 1

    print(filtering_word, nCnt)

학원 0
스터디 0
교육원 0
교육센터 0
평생교육원 0
필라테스 0
아카데미 0
병원 0
부동산 0
중개 0
건축사 0
의원 0
결혼 0
출판 0
손해사정 0
축구 0
스쿨 0
학교 0
산부인과 0
주얼리 0
부티크 0
복지관 0
스투디오 0
에스씨에이게임 0
렌트카 0
스타트업 0
의료센터 0
웨딩 0
에이치알 0
테라로사 0
성형외과 0


### 공고명에 부분적으로 포함되는 단어 필터링

In [6]:
for filtering_word in filtering_job_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
#                 print(job_list['title'])
                if job_list['status'] == 'wait':
                    job_list['status'] = 'close'
                    nCnt += 1

    print(filtering_word, nCnt)

영업관리 0
매장관리 0
고객 0
사무보조 0
마케팅 0
마케터 0
경영지원 0
NodeJs 0
Nestjs 0
부동산 0
영업사원 0
자재관리 0
회계 0
재무 0
총무 0
영상의학실 0
유지보수 0
고객응대 0
텔레마케터 0
아웃바운드 0
국비지원 0
화장품 0
디자이너 0
인플루언서 0
MD 0
AMD 0
온라인 0
네일리스트 0
웹디자인 0
기술영업 0
프론트엔드 0
회로설계 0
백엔드 0
강사 0
경리 0
사무행정 0
보험설계사 0
유튜브 0
유투브 0
국비무료 0
국비지원 0
수행기사 0
행정보조 0
구축사업 0
골프장 0
영업지원 0
Node.js 0
node.js 0
방송장비 0


In [10]:
# JSON 파일 쓰기
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()